In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs
from geo_tool import Point_Cloud
from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs
from tf_lab.point_clouds.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet
from tf_lab.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only, encoder_with_convs_and_symmetry_new
from tf_lab.nips.helper import average_per_class
from tf_lab.point_clouds.point_net_classifier import PointNetClassifier
from general_tools.in_out.basics import unpickle_data
from tf_lab.models.classifier import General_CLF

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/'

In [13]:
n_classes = 5
n_pc_points = 2048
s = str(n_classes) + '_sn_classes/'

In [11]:
mixed_train_data = '/orions4-zfs/projects/optas/DATA/OUT/icml/augmented_training_data/' + s + 'mixed_train_data'
mixed_train_data = unpickle_data(mixed_train_data).next()
pure_train_data = '/orions4-zfs/projects/optas/DATA/OUT/icml/augmented_training_data/' + s + 'pure_train_data'
pure_train_data = unpickle_data(pure_train_data).next()
pure_test_data = '/orions4-zfs/projects/optas/DATA/OUT/icml/augmented_training_data/' + s + 'pure_test_data'
pure_test_data = unpickle_data(pure_test_data).next()
pure_val_data = '/orions4-zfs/projects/optas/DATA/OUT/icml/augmented_training_data/' + s + 'pure_val_data'
pure_val_data = unpickle_data(pure_val_data).next()
syn_data = '/orions4-zfs/projects/optas/DATA/OUT/icml/augmented_training_data/'+ s + 'syn_data'
syn_data = unpickle_data(syn_data).next()

In [14]:
experiment_name = 'pure_data_' + str(n_classes) + '_sn_classes'
experiment_name = '_'.join([experiment_name, 'pts', str(n_pc_points)])
train_dir = osp.join(top_data_dir, 'OUT/exploration/nn_models/sup_clf')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir);

In [15]:
n_input = [n_pc_points, 3]

encoder = encoder_with_convs_and_symmetry_new
decoder = decoder_with_fc_only

decoder_args = {'layer_sizes': [256, 256, n_classes], 'b_norm': False}

encoder_args = {'n_filters': [64, 128, 256, 256],
                'filter_sizes': [1, 1, 1, 1],
                'strides': [1, 1, 1, 1],
                'b_norm': True
               }

conf = Conf(
            n_input = n_input,
            n_output = [n_classes],
            training_epochs = 100,
            batch_size = 512,
            train_dir = train_dir,
            loss_display_step = 1,
            saver_step = 10,
            learning_rate = 0.0005,
            z_rotate = False, 
            encoder = encoder,
            decoder = decoder,
            encoder_args = encoder_args,
            decoder_args = decoder_args
            )

conf.one_hot = False
conf.experiment_name = experiment_name
conf.save(osp.join(conf.train_dir, 'configuration'))    
reset_tf_graph()
pnet_clf = PointNetClassifier(experiment_name, conf)

In [16]:
tr_data = pure_train_data

In [17]:
c = conf
for _ in xrange(c.training_epochs):
    
    loss, duration = pnet_clf._single_epoch_train(tr_data, c)
    
    epoch = int(pnet_clf.sess.run(pnet_clf.epoch.assign_add(tf.constant(1.0))))
    print 'TRAIN epoch: %d loss: %f' % (epoch, loss)
    
    if epoch % 2 == 0:                
        avg_test_ac = General_CLF.classify_feed(pnet_clf, pure_test_data.point_clouds, 200, pure_test_data.labels)[1]        
        avg_val_ac = General_CLF.classify_feed(pnet_clf, pure_val_data.point_clouds, 200, pure_val_data.labels)[1]
        
        print 'TEST epoch: %d accuracy: %f' % (epoch, avg_test_ac)
        print 'VAL epoch: %d accuracy: %f' % (epoch, avg_val_ac)

TRAIN epoch: 1 loss: 4.086496
TRAIN epoch: 2 loss: 1.147119
TEST epoch: 2 accuracy: 0.941098
VAL epoch: 2 accuracy: 0.931864
TRAIN epoch: 3 loss: 0.164723
TRAIN epoch: 4 loss: 0.102114
TEST epoch: 4 accuracy: 0.973710
VAL epoch: 4 accuracy: 0.965932
TRAIN epoch: 5 loss: 0.089033
TRAIN epoch: 6 loss: 0.082811
TEST epoch: 6 accuracy: 0.979368
VAL epoch: 6 accuracy: 0.972612
TRAIN epoch: 7 loss: 0.077322
TRAIN epoch: 8 loss: 0.076261
TEST epoch: 8 accuracy: 0.981364
VAL epoch: 8 accuracy: 0.974616
TRAIN epoch: 9 loss: 0.074478
TRAIN epoch: 10 loss: 0.072811
TEST epoch: 10 accuracy: 0.981364
VAL epoch: 10 accuracy: 0.975284
TRAIN epoch: 11 loss: 0.069851
TRAIN epoch: 12 loss: 0.068748
TEST epoch: 12 accuracy: 0.980699
VAL epoch: 12 accuracy: 0.977288
TRAIN epoch: 13 loss: 0.068422
TRAIN epoch: 14 loss: 0.066108
TEST epoch: 14 accuracy: 0.982363
VAL epoch: 14 accuracy: 0.977956
TRAIN epoch: 15 loss: 0.064340
TRAIN epoch: 16 loss: 0.063474
TEST epoch: 16 accuracy: 0.984692
VAL epoch: 16 accu